# Get RSL dictionary
This notebook contain pipeline for transformation Russian speach language to get simple text on RSL dict


In [ ]:
#Add google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Diplom/Generation/s2t/"

## Parse RSL dictionary
Use [spreadthesign](https://www.spreadthesign.com/ru.ru/search/by-category/)

In [ ]:
#Import moduls
import requests
from bs4 import BeautifulSoup
import re
import os
from tqdm import tqdm

import pandas as pd
import numpy as np


In [ ]:
url = 'https://www.spreadthesign.com/ru.ru/search/by-category/'

In [ ]:
class load_dictionary:
    """docstring"""
 
    def __init__(self, url, main_category_nm, path_):
        """Constructor"""
        self.url = url
        self.main_category = main_category_nm
        self.path_ = path_
        self.words2dict = []
        self.part_of_speech2dict = []
        self.category2dict = []
        self.cat_url = []
        self.category = []      
    
    '''get_all_categories_url_list'''
    def get_all_cat_url_list(self):
        '''load main'''
        r  =  requests.get(self.url)
        soup  =  BeautifulSoup(r.text,"html.parser")
        
        '''get_all_cat_url_list'''
        self.cat_url = []
        for teg_li in soup.find_all('li', {"class": "js-searchable"}): 
            for teg_a in teg_li.find_all('a'): 
                self.cat_url.append("https://www.spreadthesign.com" + teg_a['href'])
                self.category.append(teg_a.text)
                break
    
    ''' Parse & add words to dict'''
    def add2dict(self,text_,cat_name__):

        text_splited = text_.split()
        words_text = ' '.join(text_splited[:-1])
        part_of_speech = text_splited[-1]

        for k in re.split(';',words_text):
            k = ' '.join(k.split())
            self.words2dict.append(k)
            self.part_of_speech2dict.append(part_of_speech)
            self.category2dict.append(cat_name__)
          
    
    ''' add_one_category_to_dict'''    
    def add_one_cat_to_dict(self, cat_one_url_, cat_name_):
        cat_r  =  requests.get(cat_one_url_)
        cat_soup = BeautifulSoup(cat_r.text,"html.parser")

        next_page = True

        while next_page:
            '''For one page with words extract them'''
            for teg_div in cat_soup.find_all('div', {"class": "search-result-title"}): 
                self.add2dict(text_ = teg_div.text, cat_name__ = cat_name_)
                

            ''' next page? get url'''
            if cat_soup.text.find('Следующая страница')>-1:
                for teg_div2 in cat_soup.find_all('div', {"class": "col-xs-4 search-pager-next"}): 
                    for teg_a2 in teg_div2.find_all('a'): 
                        next_page_url = cat_one_url_+teg_a2['href']
                        cat_r  =  requests.get(next_page_url)
                        cat_soup = BeautifulSoup(cat_r.text,"html.parser")
                    break
            else:
                next_page = False
    
    
    def load_all(self):
        self.get_all_cat_url_list()

        for i, cat_one_url in tqdm(enumerate(self.cat_url)):
            self.add_one_cat_to_dict( cat_one_url_ = cat_one_url, cat_name_ = self.category[i])
            
     
    def create_dict(self):
        '''download'''
        self.load_all()

        '''dataframe'''
        self.RSL_df = pd.DataFrame({'WORD':self.words2dict,
                        'PART_OF_SPEECH':self.part_of_speech2dict,
                        'TOPIC_CATEGORY':self.category2dict})
        
        #Fix bag
        cond = self.RSL_df['PART_OF_SPEECH'] == 'фарш'

        self.RSL_df.loc[cond,'WORD'] = self.RSL_df.loc[cond,'PART_OF_SPEECH']
        self.RSL_df.loc[cond,'PART_OF_SPEECH'] = 'Существительное' 

        # Drop duplicates
        self.RSL_df.drop_duplicates(inplace= True)

        #размечаем строки, где топик = общая катеория (особенность сайта)
        self.RSL_df['MAIN_CATEGORY_F'] = [1 if t in self.main_category else 0 for t in self.RSL_df['TOPIC_CATEGORY']]

        #Убираем повторы за счет исключения определения слов из более обобщенной категории. См. "автомобиль", например
        self.RSL_df['Rank'] = self.RSL_df.sort_values(['MAIN_CATEGORY_F'], ascending=[True]) \
                    .groupby(['WORD','PART_OF_SPEECH']) \
                    .cumcount() + 1                   
        self.RSL_df = self.RSL_df[(self.RSL_df['MAIN_CATEGORY_F'] == 0) | (self.RSL_df['Rank'] == 1)]


    def save_df(self):
        self.RSL_df.to_csv(self.path_ + "RSL_dict_clear.csv")

In [ ]:
main_category = ["Жестовый язык для начинающих",
"Общие положения",
"Предложения",
"Религия",
"Педагогика",
"Язык",
"Искусство и развлечения",
"Социальные исследования" ,
"География и путешествия",
"Еда и напитки",
"Стиль жизни",
"Спорт и Отдых",
"Технология",
"Компьютер и современные технологии",
"Наука",
"Здоровье и медицина",
"Жесты малышей"]

In [ ]:
RSL_dict = load_dictionary(url = url,main_category_nm= main_category,path_= path)

In [ ]:
RSL_dict.create_dict()


0it [00:00, ?it/s]
1it [00:03,  3.60s/it]
2it [00:14,  5.84s/it]
3it [00:15,  4.26s/it]
4it [00:17,  3.69s/it]
5it [00:20,  3.42s/it]
6it [00:22,  3.10s/it]
7it [00:24,  2.72s/it]
8it [00:26,  2.55s/it]
9it [00:27,  2.00s/it]
10it [00:27,  1.54s/it]
11it [00:28,  1.22s/it]
12it [00:28,  1.01s/it]
13it [00:33,  2.08s/it]
14it [00:34,  1.62s/it]
15it [00:34,  1.33s/it]
16it [00:38,  1.94s/it]
17it [00:38,  1.52s/it]
18it [00:46,  3.34s/it]
19it [00:47,  2.65s/it]
20it [00:47,  2.01s/it]
21it [00:54,  3.29s/it]
22it [01:25, 11.87s/it]
23it [01:26,  8.45s/it]
24it [01:27,  6.30s/it]
25it [01:29,  4.94s/it]
26it [01:29,  3.61s/it]
27it [01:40,  5.62s/it]
28it [01:46,  5.93s/it]
29it [01:47,  4.31s/it]
30it [01:57,  6.13s/it]
31it [01:59,  4.93s/it]
32it [02:00,  3.59s/it]
33it [02:01,  2.81s/it]
34it [02:01,  2.12s/it]
35it [02:15,  5.57s/it]
36it [02:17,  4.44s/it]
37it [02:18,  3.42s/it]
38it [02:19,  2.87s/it]
39it [02:23,  3.03s/it]
40it [02:24,  2.62s/it]
41it [02:26,  2.16s/it]
42it 

In [ ]:
RSL_dict.RSL_df

,WORD,PART_OF_SPEECH,TOPIC_CATEGORY,MAIN_CATEGORY_F,Rank
1218,бежевый,Существительное,Цвета,0,1
1219,белокурый,Прилагательное,Цвета,0,2
1220,белый,Существительное,Цвета,0,1
1221,бледный,Прилагательное,Цвета,0,1
1222,бронза,Существительное,Цвета,0,2
...,...,...,...,...,...
29529,чистота,Существительное,Общие сведения: Здоровье,0,2
29530,чистый,Прилагательное,Общие сведения: Здоровье,0,2
29531,шампунь,Существительное,Общие сведения: Здоровье,0,2
29532,щетка-чесалка на длинной ручке,Существительное,Общие сведения: Здоровье,0,1


In [ ]:
RSL_dict.save_df()

In [ ]:
RSL_dict_df = RSL_dict.RSL_df

In [ ]:
RSL_dict_df = pd.read_csv(path + "RSL_dict_clear.csv", index_col= 0)
RSL_dict_df.head()

,WORD,PART_OF_SPEECH,TOPIC_CATEGORY,MAIN_CATEGORY_F,Rank
1218,бежевый,Существительное,Цвета,0,1
1219,белокурый,Прилагательное,Цвета,0,2
1220,белый,Существительное,Цвета,0,1
1221,бледный,Прилагательное,Цвета,0,1
1222,бронза,Существительное,Цвета,0,2


## Add synonyms/hyponyms/etc.

[ruwordnet](https://github.com/avidale/python-ruwordnet)

[wiki-ru-wordnet](https://wiki-ru-wordnet.readthedocs.io/en/latest/)

In [ ]:
!pip install ruwordnet
!ruwordnet download

!pip install wiki_ru_wordnet

!pip install pymorphy2
!pip install pymorphy2-dicts
!pip install 'DAWG-Python >= 0.7'

  Created wheel for ruwordnet: filename=ruwordnet-0.0.2-cp37-none-any.whl size=7398 sha256=37c3d638fc050350bbb7f44abc76baa945beebc8c3113df80f5f6fbefa6b6020
  Stored in directory: /root/.cache/pip/wheels/ef/f0/c0/555ac9b4c7d28ff6be41fb5f1e0de97894168e59eb01cf8159
Successfully built ruwordnet
downloading a ruwordnet model from https://github.com/avidale/python-ruwordnet/releases/download/0.0.2/ruwordnet.db
     |████████████████████████████████| 20.6MB 1.4MB/s 
  Created wheel for wiki-ru-wordnet: filename=wiki_ru_wordnet-1.0.3-cp37-none-any.whl size=20890443 sha256=59148bdfeeb4c5107bacb2ed4ece5199650d72aec37bdf25729f6d54cbfee519
  Stored in directory: /root/.cache/pip/wheels/48/6a/c8/22758a66c35b21c1491861e8ab4809b1fd466d1d2445e99fda
Successfully built wiki-ru-wordnet
     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 8.2MB 7.6MB/s 
     |████████████████████████████████| 7.1MB 5.4MB/s 


In [ ]:
from ruwordnet import RuWordNet
from wiki_ru_wordnet import WikiWordnet

import pymorphy2

In [ ]:
def flatten(d):
    v = [[i] if not isinstance(i, list) else flatten(i) for i in d]
    return [i for b in v for i in b]

def add_onyms(word_):
    '''init'''
    candidates_list = []
    morph = pymorphy2.MorphAnalyzer()
    word_ = morph.parse(word_.lower())[0].normal_form

    wn = RuWordNet()

    wikiwordnet = WikiWordnet()
    synsets = wikiwordnet.get_synsets(word_)

    '''Dict 1'''
    #synonyms'''
    candidates_list.append([h.name.lower().split(' (')[0].split(', ')  for s in wn.get_senses(word_) for h in s.synset.senses])
    #hyponyms'''
    candidates_list.append([h.title.lower().split(' (')[0].split(', ')  for s in wn.get_senses(word_) for h in s.synset.hyponyms])
    #holonyms'''
    candidates_list.append([h.title.lower().split(' (')[0].split(', ')  for s in wn.get_senses(word_) for h in s.synset.holonyms])
    #conclusions'''
    candidates_list.append([h.title.lower().split(' (')[0].split(', ')  for s in wn.get_senses(word_) for h in s.synset.conclusions])
    #pos_synonyms'''
    candidates_list.append([h.title.lower().split(' (')[0].split(', ')  for s in wn.get_senses(word_) for h in s.synset.pos_synonyms])
  

    '''Dict 2'''
    #synonyms'''
    candidates_list.append([h.lemma().lower().split(', ')  for s in synsets for h in s.get_words()])
    #hyponyms'''
    candidates_list.append([w.lemma().lower().split(', ')  for s in synsets for h in wikiwordnet.get_hyponyms(s) for w in h.get_words()])
    

    candidates_list = list(set(flatten(d  =candidates_list)))
    candidates_for_df = [[word_]*len(candidates_list), candidates_list]
    return candidates_for_df

In [ ]:
syn = np.empty(shape = (2,0), dtype='object')

for w in tqdm(set(RSL_dict_df['WORD'])):
    syn = np.hstack((syn,add_onyms(w)))

# RSL_dict_syn = RSL_dict_df.copy()
# RSL_dict_syn = RSL_dict_syn.merge(add_onyms('красивый'),on =['WORD_LOWERCASE'], how='left')

100%|██████████| 12145/12145 [7:02:13<00:00,  2.09s/it]


In [ ]:
(pd.DataFrame(syn.T)).to_csv(path + "syn_dict.csv")

In [ ]:
save = syn.copy()

In [ ]:
temp = pd.DataFrame(syn.T)
temp.columns = ['WORD_LOWERCASE','ONYM']

In [ ]:
temp.head()

,WORD_LOWERCASE,ONYM
0,грузовик,автофургон
1,грузовик,грузовик
2,грузовик,автосамосвал
3,грузовик,автомобильная цистерна
4,грузовик,пятитонка


In [ ]:
RSL_dict_df['WORD_LOWERCASE'] = RSL_dict_df['WORD'].apply(lambda x: x.lower())

In [ ]:
RSL_dict_df.merge(temp,on =['WORD_LOWERCASE'], how='left').drop_duplicates().to_csv(path + 'RSL_ONYMS.csv')


In [ ]:
# RSL_dict_syn = RSL_dict_df.copy()
# RSL_dict_syn = RSL_dict_syn.merge(syn,on =['WORD_LOWERCASE'], how='left')

In [ ]:
RSL_dict_df.merge(temp,on =['WORD_LOWERCASE'], how='left').head(50)

,WORD,PART_OF_SPEECH,TOPIC_CATEGORY,MAIN_CATEGORY_F,Rank,WORD_LOWERCASE,ONYM
0,бежевый,Существительное,Цвета,0,1,бежевый,бежевый цвет
1,бежевый,Существительное,Цвета,0,1,бежевый,бежевый
2,бежевый,Существительное,Цвета,0,1,бежевый,плавый
3,белокурый,Прилагательное,Цвета,0,2,белокурый,светловолосый
4,белокурый,Прилагательное,Цвета,0,2,белокурый,белобрысый
5,белокурый,Прилагательное,Цвета,0,2,белокурый,белокурый
6,белокурый,Прилагательное,Цвета,0,2,белокурый,блондинистый
7,белый,Существительное,Цвета,0,1,белый,беленький
8,белый,Существительное,Цвета,0,1,белый,европеец
9,белый,Существительное,Цвета,0,1,белый,белый человек
